# CP421 Project - Recommendation System(CF)
#### Shawn Davis 
#### Adam Cassidy 
#### Mengdan Wan

##### Import rating and movie data

In [1]:
import pandas as pd
import numpy as np

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('ml-latest-small/ml-latest-small/ratings.csv', skiprows=1, sep=',', header=None, names=rnames)

ratings[:]


,user_id,movie_id,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [2]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('ml-latest-small/ml-latest-small/movies.csv', skiprows=1, sep=',', header=None, names=mnames,)

movies

,movie_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


##### Merge movies and ratings

In [3]:
data = pd.merge(ratings, movies)
data

,user_id,movie_id,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller


##### Random split data for training and test (train = 0.9 and test = 0.1)

In [4]:
data_copy = data.copy()
train_set = data_copy.sample(frac=0.9) #random choose sample (change the frac to change the training ratio)
test_set = data_copy.drop(train_set.index) #Drop the training data, the rest is test data

train_set[:]

,user_id,movie_id,rating,timestamp,title,genres
89002,414,7439,2.0,1089043902,"Punisher, The (2004)",Action|Crime|Thriller
20756,496,912,4.5,1415166610,Casablanca (1942),Drama|Romance
63046,571,3072,5.0,966900054,Moonstruck (1987),Comedy|Romance
24778,4,4252,3.0,1007569465,"Circle, The (Dayereh) (2000)",Drama
22466,135,2076,4.0,1009693935,Blue Velvet (1986),Drama|Mystery|Thriller
...,...,...,...,...,...,...
26542,284,410,3.0,832786349,Addams Family Values (1993),Children|Comedy|Fantasy
66600,590,3638,2.5,1258428713,Moonraker (1979),Action|Adventure|Sci-Fi|Thriller
67586,341,59900,5.0,1487275987,You Don't Mess with the Zohan (2008),Comedy
49258,18,1260,4.0,1455059781,M (1931),Crime|Film-Noir|Thriller


In [5]:
#Create a data frame for movies and genres in training data, use for content filter
train_mov = pd.DataFrame(train_set.loc[:,'title':'genres'])
train_no_dup = train_mov.drop_duplicates('title')
train_no_dup = train_no_dup.reset_index(drop=True)
train_no_dup

,title,genres
0,"Punisher, The (2004)",Action|Crime|Thriller
1,Casablanca (1942),Drama|Romance
2,Moonstruck (1987),Comedy|Romance
3,"Circle, The (Dayereh) (2000)",Drama
4,Blue Velvet (1986),Drama|Mystery|Thriller
...,...,...
9358,All the Vermeers in New York (1990),Comedy|Drama|Romance
9359,"Day of the Locust, The (1975)",Drama
9360,Gigli (2003),Comedy|Crime|Romance
9361,UnHung Hero (2013),Documentary


In [6]:
#Covert all the movies genres to binary data, 1 if the movie belong the genre, 0 otherwise
GENRES = ["Action", "Adventure", "Animation" ,"Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
            "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western",
              "(no genres listed)"]

#Create the matrix for genres of all the training movies
genres_matrix = []
for i in range(0, len(train_no_dup)):
    b = []
    mov = train_no_dup.loc[i, 'genres']
    for g in GENRES:
        if g in mov:
            b.append(1)
        else:
            b.append(0)
    print(b)
    genres_matrix.append(b) #Making 2D array
print(genres_matrix)

[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1

[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 1

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0

[[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 

In [7]:
#Create dataframe for the 2D array for binary matrix of genres and movies
gen = pd.DataFrame(genres_matrix, index = train_no_dup.loc[:, 'title'], columns = GENRES)
gen = gen.T
gen

title,"Punisher, The (2004)",Casablanca (1942),Moonstruck (1987),"Circle, The (Dayereh) (2000)",Blue Velvet (1986),Logan's Run (1976),"Others, The (2001)",Apollo 13 (1995),Alive (1993),"Bourne Identity, The (2002)",...,Tomboy (2011),American Buffalo (1996),Born to Be Wild (1995),Glory Daze (1995),"Muppet Christmas: Letters to Santa, A (2008)",All the Vermeers in New York (1990),"Day of the Locust, The (1975)",Gigli (2003),UnHung Hero (2013),People Will Talk (1951)
Action,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Adventure,0,0,0,0,0,1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
Animation,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Children,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
Comedy,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,1,0,1,0,1
Crime,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
Documentary,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Drama,0,1,0,1,1,0,1,1,1,0,...,1,1,1,1,0,1,1,0,0,0
Fantasy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Film-Noir,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#Calculating the similarties by using genres
corr_gen = gen.corr(method = 'pearson')
corr_gen

title,"Punisher, The (2004)",Casablanca (1942),Moonstruck (1987),"Circle, The (Dayereh) (2000)",Blue Velvet (1986),Logan's Run (1976),"Others, The (2001)",Apollo 13 (1995),Alive (1993),"Bourne Identity, The (2002)",...,Tomboy (2011),American Buffalo (1996),Born to Be Wild (1995),Glory Daze (1995),"Muppet Christmas: Letters to Santa, A (2008)",All the Vermeers in New York (1990),"Day of the Locust, The (1975)",Gigli (2003),UnHung Hero (2013),People Will Talk (1951)
title,,,,,,,,,,,,,,,,,,,,,
"Punisher, The (2004)",1.000000,-0.148522,-0.148522,-0.102062,0.208333,0.208333,0.130437,-0.148522,-0.102062,0.604167,...,-0.102062,0.321798,-0.223607,-0.102062,-0.148522,-0.187500,-0.102062,0.208333,-0.102062,-0.148522
Casablanca (1942),-0.148522,1.000000,0.441176,0.687184,0.321798,-0.148522,0.243544,0.441176,0.687184,-0.148522,...,0.687184,0.441176,0.243544,0.687184,-0.117647,0.792118,0.687184,0.321798,-0.080845,0.441176
Moonstruck (1987),-0.148522,0.441176,1.000000,-0.080845,-0.148522,-0.148522,-0.177123,-0.117647,-0.080845,-0.148522,...,-0.080845,-0.117647,0.243544,-0.080845,0.441176,0.792118,-0.080845,0.792118,-0.080845,1.000000
"Circle, The (Dayereh) (2000)",-0.102062,0.687184,-0.080845,1.000000,0.544331,-0.102062,0.456435,0.687184,1.000000,-0.102062,...,1.000000,0.687184,0.456435,1.000000,-0.080845,0.544331,1.000000,-0.102062,-0.055556,-0.080845
Blue Velvet (1986),0.208333,0.321798,-0.148522,0.544331,1.000000,-0.187500,0.838525,0.321798,0.544331,0.604167,...,0.544331,0.321798,0.130437,0.544331,-0.148522,0.208333,0.544331,-0.187500,-0.102062,-0.148522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
All the Vermeers in New York (1990),-0.187500,0.792118,0.792118,0.544331,0.208333,-0.187500,0.130437,0.321798,0.544331,-0.187500,...,0.544331,0.321798,0.484481,0.544331,0.321798,1.000000,0.544331,0.604167,-0.102062,0.792118
"Day of the Locust, The (1975)",-0.102062,0.687184,-0.080845,1.000000,0.544331,-0.102062,0.456435,0.687184,1.000000,-0.102062,...,1.000000,0.687184,0.456435,1.000000,-0.080845,0.544331,1.000000,-0.102062,-0.055556,-0.080845
Gigli (2003),0.208333,0.321798,0.792118,-0.102062,-0.187500,-0.187500,-0.223607,-0.148522,-0.102062,-0.187500,...,-0.102062,0.321798,0.130437,-0.102062,0.321798,0.604167,-0.102062,1.000000,-0.102062,0.792118


In [9]:
#include global mean, min, max of training data
train_set.describe()

,user_id,movie_id,rating,timestamp
count,90752.000000,90752.000000,90752.000000,9.075200e+04
mean,326.013465,19431.697175,3.500325,1.206156e+09
std,182.494968,35522.193924,1.041263,2.161245e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1198.750000,3.000000,1.019125e+09
50%,325.000000,2993.000000,3.500000,1.186161e+09
75%,477.000000,8128.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537757e+09


##### Get mean rating of movies

In [10]:
#Get the mean for every training movies
ratings = pd.DataFrame(train_set.groupby('title')['rating'].mean())
ratings

,rating
title,
'71 (2014),4.000000
'Hellboy': The Seeds of Creation (2004),4.000000
'Round Midnight (1986),3.500000
'Salem's Lot (2004),5.000000
'Til There Was You (1997),4.000000
...,...
anohana: The Flower We Saw That Day - The Movie (2013),3.000000
eXistenZ (1999),3.763158
xXx (2002),2.736842


##### Count how many ratings of each movie

In [11]:
#Get the count of how many times rated by users for every training movies
ratings['number_of_ratings'] = data.groupby('title')['rating'].count()
ratings.sort_values(by='number_of_ratings', ascending=False).head(10)

,rating,number_of_ratings
title,,
Forrest Gump (1994),4.159649,329
"Shawshank Redemption, The (1994)",4.441281,317
Pulp Fiction (1994),4.190217,307
"Silence of the Lambs, The (1991)",4.164683,279
"Matrix, The (1999)",4.170683,278
Star Wars: Episode IV - A New Hope (1977),4.221973,251
Jurassic Park (1993),3.734234,238
Braveheart (1995),3.997561,237
Terminator 2: Judgment Day (1991),3.967662,224


In [12]:
#import seaborn as sns
#sns.jointplot(x='rating', y='number_of_ratings', data=ratings)

In [13]:
#Create the pivot table for movies and users, rating as the value
train_matrix = pd.pivot_table(train_set, values='rating', index='user_id', columns='title')
train_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...,Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN


##### Get pearson correlation matrix of movies

In [14]:
#Calculate the pearson correlations between all the pair of training movies
corr_matrix = train_matrix.corr(method = 'pearson')
corr_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...,Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
anohana: The Flower We Saw That Day - The Movie (2013),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
eXistenZ (1999),NaN,NaN,NaN,NaN,NaN,-0.067522,NaN,0.737043,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,-0.337963
xXx (2002),NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.817105,NaN,NaN,...,NaN,NaN,-0.922613,NaN,NaN,NaN,1.000000,1.000000,0.981981,0.500000


## The Recommendation System
#### Top 10 recommendation movies which every movies are rated at least N times by users.

In [15]:
N = 20
#some movies only has negtive similarities with other movies, set threshold to -1 to see the result
similarity_Threshold = 0    
movie_i = input("Enter the movie name: ") #Get movie name as input

#Find similar movie according to past rating by users (CF item-based)
if (corr_matrix.index == movie_i).any():   #recommend by collaborative filtering
    recom = pd.DataFrame(corr_matrix[movie_i])
    recom.dropna(inplace=True)
    recom = recom.join(ratings['number_of_ratings'])  
    recom = recom[recom['number_of_ratings'] > N] #filtering the similar movies that at least rated N times
    #recom = recom[recom[movie_i] < 1] 
    recom = recom[recom[movie_i] > similarity_Threshold] #filtering by the threshold of similarities
    recom_n = recom.sort_values(by=movie_i, ascending=False).head(10)
    print(recom_n.empty)
    print("1")
    
    #Only filter by past rating, some movies may not find the similar movie.
    #in this case, we find similar movie by using content filtering of genres.
    if recom_n.empty:   #recommend by content filtering using genres
        recom = pd.DataFrame(corr_gen[movie_i])
        recom.dropna(inplace=True)
        recom = recom.join(ratings['number_of_ratings'])
        recom = recom[recom['number_of_ratings'] > N]
        #recom = recom[recom[movie_i] < 1] 
        recom = recom[recom[movie_i] > similarity_Threshold] 
        recom_n = recom.sort_values(by=movie_i, ascending=False).head(10)
        print(recom_n.empty)
        print("2")
        
#Another case is for the new movie, the movie did not in training movies
else:
    
    # if the movie already add to training movie then directly using similarties to get similar movies
    if (corr_gen.index == movie_i).any():   #recommend by content filtering using genres
        recom = pd.DataFrame(corr_gen[movie_i])
        recom.dropna(inplace=True)
        recom = recom.join(ratings['number_of_ratings'])
        recom = recom[recom['number_of_ratings'] > N]
        #recom = recom[recom[movie_i] < 1] 
        recom = recom[recom[movie_i] > similarity_Threshold]  
        recom_n = recom.sort_values(by=movie_i, ascending=False).head(10)
        print(recom_n.empty)
        print("3")
    #calculate the similarty between new movie and the movies already in traing movie
    else:    #recommend by content filtering using genres, for new movies.
        new_g = []
        b = []
        for genre in GENRES:
            if genre in g:
                b.append(1)
            else:
                b.append(0)
        new_g.append(b)
        current = pd.DataFrame(new_g, index=[movie_i],columns=GENRES)
        current = current.T
        
        # add new movie to traing movies
        gen = gen.join(current)

        corr_gen = gen.corr(method = 'pearson')
        recom = pd.DataFrame(corr_gen[movie_i])
        recom.dropna(inplace=True)
        recom = recom.join(ratings['number_of_ratings'])
        recom = recom[recom['number_of_ratings'] > N]
        #recom = recom[recom[movie_i] < 1] 
        recom = recom[recom[movie_i] > similarity_Threshold] 
        recom_n = recom.sort_values(by=movie_i, ascending=False).head(10)
        print(recom_n.empty)
        print("4")
        
recom_n



Enter the movie name: 'Hellboy': The Seeds of Creation (2004)
True
1
False
2


,'Hellboy': The Seeds of Creation (2004),number_of_ratings
title,,
Pirates of the Caribbean: The Curse of the Black Pearl (2003),0.864099,149
Big Trouble in Little China (1986),0.864099,28
Last Action Hero (1993),0.864099,53
Pirates of the Caribbean: At World's End (2007),0.864099,56
"Princess Bride, The (1987)",0.728571,142
"Goonies, The (1985)",0.728571,57
Army of Darkness (1993),0.728571,51
Hancock (2008),0.728571,29
Dogma (1999),0.724569,79


## Making prediction of test data

In [16]:
#drop the useless columns in test data
test_set.drop(['movie_id', 'timestamp'], axis=1,inplace=True)
test_set

,user_id,rating,title,genres
9,31,5.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
25,73,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
34,96,5.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
43,132,2.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
49,144,3.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...
100783,610,3.0,Escape From Tomorrow (2013),Drama|Fantasy|Horror
100798,610,4.0,"Guest, The (2014)",Thriller
100831,610,2.5,Bloodmoon (1997),Action|Thriller
100832,610,4.5,Sympathy for the Underdog (1971),Action|Crime|Drama


In [17]:
print(len(test_set))

#convert data frame to numpy array, that can go through every test data 
test_array = test_set.to_numpy()
test_array

10084


array([[31, 5.0, 'Toy Story (1995)',
        'Adventure|Animation|Children|Comedy|Fantasy'],
       [73, 4.5, 'Toy Story (1995)',
        'Adventure|Animation|Children|Comedy|Fantasy'],
       [96, 5.0, 'Toy Story (1995)',
        'Adventure|Animation|Children|Comedy|Fantasy'],
       ...,
       [610, 2.5, 'Bloodmoon (1997)', 'Action|Thriller'],
       [610, 4.5, 'Sympathy for the Underdog (1971)',
        'Action|Crime|Drama'],
       [610, 3.0, 'Hazard (2005)', 'Action|Drama|Thriller']], dtype=object)

##### Go through every test data, and using pearson correlation matrix to find top k similar movies that already rated by current user. 
##### If user did not rate any similar movie, then use the average rating as predicting of this movie
##### If no similar movie or the movie did not in trainning data(New movies), then using 'Content Filtering' by genres to find the similar movies, and make predicting by using average rating of the top 20 similar movies which filtering by genres.

In [18]:
pred = [] # use for measure RMSE
k = 20
c=0
noOfRatingThreshold = 20

for m in test_array:   #loop for go through every test data
    t = m[2]
    u = m[0]
    g = m[3]
    
    #Select similar movies as in part of recomendation system
    if (corr_matrix.index == t).any():
        movie_corr = pd.DataFrame(corr_matrix[t])
        movie_corr.dropna(inplace=True)
        #movie_corr = movie_corr.join(ratings['number_of_ratings'])
        #movie_corr = movie_corr[movie_corr['number_of_ratings'] > noOfRatingThreshold]
        movie_corr = movie_corr[movie_corr[t] < 1] 
        movie_corr = movie_corr[movie_corr[t] > 0] 
        sim_movies = movie_corr.sort_values(by=t, ascending=False)
        #sim_movies
        sim_title = sim_movies.index
        #print(sim_title)

        #calculate the prediction rating
        count = 0
        r_s = 0
        total_sim = 0
        for movie in sim_title:
            r = train_matrix.loc[u, movie]
            #print(u)
            #print(movie)
            if not np.isnan(r):  
                #print(r)
                count += 1
                sim = sim_movies.loc[movie, t]
                r_s += r*sim   #sum of (rating * similarity)
                total_sim += sim  #sum of similarity
            if count >= k: #choose first 20 similar movies that has rated by the user
                break
                #print(count)
        
        if total_sim == 0: #this is case of no similar movies in collabortive filtering
            
            g_corr = pd.DataFrame(corr_gen[t])
            g_corr.dropna(inplace=True)
            #movie_corr = movie_corr.join(ratings['number_of_ratings'])
            #movie_corr = movie_corr[movie_corr['number_of_ratings'] > noOfRatingThreshold]
            g_corr = g_corr[g_corr[t] < 1] 
            g_corr = g_corr[g_corr[t] > 0] 
            sim_gen = g_corr.sort_values(by=t, ascending=False)
            sim_genres = sim_gen.index
            
            count_1 = 0
            r_s_1 = 0
            total_sim_1 = 0
            for tit in sim_genres:
                if (corr_matrix.index == tit).any():
                    rt = train_matrix.loc[u, tit]
                    
                    if not np.isnan(rt):
                        count_1 += 1
                        sim1 = sim_gen.loc[tit, t]
                        #print(sim)
                        r_s_1 += rt*sim1
                        total_sim_1 += sim1
                    if count_1 >= k:
                        break
                        #print(count)
            #In this case the movie not have positive similarities with other movies
            if total_sim_1 == 0:  
                rate = ratings.loc[tit, 'rating'] #using mean rating of this movie as predicting
                
            else:
                rate = r_s_1/total_sim_1  #predicting rating = sum of (rating * similarity) / sum of similarity
                
            pred.append(rate) 
            print(rate)
            c += 1
            
        else:
            rate = r_s/total_sim  #predicting rating = sum of (rating * similarity) / sum of similarity
            pred.append(rate)
            print(rate)
            c += 1
    
    # this part for new movie
    else:  
        if (corr_gen.index == t).any():
            curr_corr = pd.DataFrame(corr_gen[t])
            curr_corr.dropna(inplace=True)
            #movie_corr = movie_corr.join(ratings['number_of_ratings'])
            #movie_corr = movie_corr[movie_corr['number_of_ratings'] > noOfRatingThreshold]
            curr_corr = curr_corr[curr_corr[t] < 1] 
            curr_corr = curr_corr[curr_corr[t] > 0] 
            sim_gen = curr_corr.sort_values(by=t, ascending=False)
            sim_tit = sim_gen.index
        else:
            #print("new")
            g_a = []
            b = []
            for genre in GENRES:
                if genre in g:
                    b.append(1)
                else:
                    b.append(0)
            #print(b)
            g_a.append(b)
            current = pd.DataFrame(g_a, index=[t],columns=GENRES)
            current = current.T
            current

            gen = gen.join(current)
            corr_gen = gen.corr(method = 'pearson')
            curr_corr = pd.DataFrame(corr_gen[t])
            curr_corr.dropna(inplace=True)
            #movie_corr = movie_corr.join(ratings['number_of_ratings'])
            #movie_corr = movie_corr[movie_corr['number_of_ratings'] > noOfRatingThreshold]
            curr_corr = curr_corr[curr_corr[t] < 1] 
            curr_corr = curr_corr[curr_corr[t] > 0] 
            sim_gen = curr_corr.sort_values(by=t, ascending=False)
            sim_tit = sim_gen.index

        count_2 = 0
        r_s_2 = 0
        tot_r = 0

        for tit in sim_tit:
            if (ratings.index == tit).any():
                rt2 = ratings.loc[tit, 'rating']
                count_2 += 1
                tot_r += rt2
            if count_2 >= 20:
                break
                
        
        if count_2 == 0:    #No genres, set the rate to 0
            rate = 0
        else:
            rate = tot_r/count_2 #new movie are in the test set has no ratings, 
                                    #so use top 20 simlary movies' average rating to get the average as predicting
        print(rate)
        pred.append(rate)
        
        #print("No Recommendation(New movie)")
        c += 1
    print(c)
print("Total movies: %d" % c)
print(len(pred))



3.686955526989126
1
3.6763228025719448
2
3.4898117407760667
3
2.8053413590238208
4
3.9056507240130633
5
3.668935899812337
6
4.760680305138938
7
3.5760232822831597
8
3.4725054113638967
9
3.550241949560414
10
3.7784261414230897
11
3.5039013870510867
12
3.6482448830350163
13
3.415108559038859
14
3.6028079081892748
15
3.802565134221221
16
3.099693696278877
17
3.7071280104394972
18
4.158615317616055
19
2.9807268546228
20
3.2603673948412326
21
3.3120749562915104
22
3.406748220913814
23
2.3684320503685967
24
4.101828363038404
25
2.6889778243824143
26
2.975477488547423
27
3.3853289044740595
28
3.5843663754364887
29
3.1129497578854486
30
3.67945731486964
31
3.743655497063215
32
4.2541022108632385
33
3.4437061297815927
34
4.347090422355134
35
3.596188992160123
36
2.902189170822071
37
2.4796665671630693
38
3.02824185043783
39
4.713563328896411
40
3.088429647479826
41
3.29297455824025
42
3.5477306906650594
43
3.678631545583569
44
3.752531260039961
45
3.176505375734846
46
3.10385243916689
47
4.2926

3.91578567289882
387
3.7618420197355893
388
3.8192474454265986
389
2.9268000415211706
390
4.120269596027044
391
3.4299107083105573
392
3.2610440533897855
393
2.9152639848783313
394
3.46388869952186
395
3.0182379921782125
396
3.385016634745445
397
2.5036487268535144
398
4.40733039683285
399
3.025794442217428
400
3.8427513348308966
401
2.7936158762294707
402
4.198996390448859
403
2.6676394699456147
404
3.427153353586227
405
3.9150642770285895
406
4.039162208487516
407
3.4657346970419094
408
3.9870204421760453
409
3.169949371575386
410
3.548746508576493
411
3.0514082938612814
412
3.5824927117436673
413
2.6409746806716567
414
4.366711658621148
415
4.157915956190212
416
3.3369100183744353
417
4.192500657515398
418
4.182506017960556
419
4.458591364769576
420
3.612829753756729
421
3.8001703617554767
422
4.144967625789381
423
3.62288560477534
424
3.7252307269032476
425
3.0925953251842238
426
2.6270046504090816
427
3.9783264433515573
428
2.807478914714051
429
3.1207535413266365
430
3.9347423422

4.43139327738932
758
3.59948894985083
759
4.318229142496463
760
3.228592855854994
761
2.912121232961395
762
3.7048747432626232
763
3.366017934040208
764
3.842915141718451
765
3.700638756365973
766
3.1171299798839116
767
2.8252111611145043
768
3.562813530158888
769
2.6705378790375423
770
3.5699802491323407
771
4.102283150424015
772
3.2646671959011915
773
3.5087524227228912
774
3.8364886643729945
775
2.9543655708564414
776
3.2221574683423837
777
3.0475578413444557
778
4.374173034171101
779
4.6685490287592275
780
2.8433723992587177
781
3.655779442265059
782
4.020805084451047
783
2.899321707665966
784
3.872388985386215
785
2.790244113679582
786
3.343595938820113
787
4.126348437459369
788
3.8972404215086467
789
3.452954108530137
790
3.9963818109840403
791
3.8240240808331234
792
4.482413597047306
793
3.276376847336764
794
2.809691933025423
795
4.106783057213994
796
3.880540489565276
797
3.7101717807370096
798
3.9717435844660582
799
3.498387096289511
800
3.7628192245568077
801
3.3791076720969

3.608193698850995
1132
3.295568374132348
1133
2.6000903143427596
1134
4.059593836545139
1135
3.983463962816672
1136
2.5060213264417754
1137
1.960278046744431
1138
3.504227314772859
1139
3.878861087518656
1140
2.559839634860488
1141
4.289356328624135
1142
3.9877920572289027
1143
4.303162473234986
1144
3.740752036202453
1145
3.7163993706458447
1146
3.196583959237818
1147
3.69316328090452
1148
4.244151924790976
1149
3.651467838385647
1150
3.536541583952006
1151
3.2318631845139523
1152
2.8437342069971163
1153
3.203013783488598
1154
4.192004983160085
1155
2.279548597720161
1156
3.751945389676729
1157
3.1057463465900574
1158
3.3526228726425624
1159
3.747665139932719
1160
3.8087710821252094
1161
2.567296178744092
1162
3.4898983418044995
1163
3.4326714815147845
1164
2.85387210355978
1165
2.5966030564316993
1166
4.177786566209708
1167
2.6666315027355343
1168
3.2533810656271203
1169
3.9654738754244896
1170
3.2540054609595606
1171
2.918560953003017
1172
2.4421566133820907
1173
3.0362942004053717


3.0969608844348304
1493
3.8206091425863153
1494
3.3936258507432044
1495
3.212265331555531
1496
3.0799334472157702
1497
3.6137652559188163
1498
3.700151764329303
1499
3.7997998782540736
1500
3.750447925103922
1501
3.4567154789454606
1502
4.365747632208398
1503
4.07063806937885
1504
3.4543410031509167
1505
3.803295227922933
1506
2.744654416805019
1507
3.944657517926976
1508
3.6106636194244723
1509
2.7904711360432466
1510
3.6168179956034656
1511
3.3240556196612747
1512
2.7353632050908048
1513
4.320714292877795
1514
3.3069369106294686
1515
3.90323620454441
1516
3.6329981543597443
1517
3.0944711925103006
1518
2.7431459764333663
1519
4.2056363869822135
1520
2.8885246337681645
1521
2.2496123638683065
1522
3.170054747240556
1523
4.157451482148533
1524
3.2973052337023367
1525
2.7883202087703163
1526
3.7077368698072886
1527
3.235950524677078
1528
3.4402484295016604
1529
3.6392356535379493
1530
2.9984418085155764
1531
3.591281130570407
1532
3.706013372148157
1533
4.141694424128311
1534
3.57788203

...

3.679752623918582
8356
2.4411015758738954
8357
3.2576247456439424
8358
3.6535843616219914
8359
3.9954882765038167
8360
3.270846418465383
8361
2.7072030224806727
8362
3.270917014387449
8363
3.954384878046051
8364
3.2750000000000012
8365
4.348928379028322
8366
3.651677017547533
8367
3.2299402214464994
8368
4.049878143663477
8369
3.542114709235685
8370
3.7446926200008837
8371
3.5757818703948425
8372
3.2750000000000012
8373
3.9750000000000005
8374
3.3722343262649352
8375
2.0214108289099197
8376
3.8957305463693244
8377
4.089113924201491
8378
3.276535422526412
8379
3.488193187141529
8380
3.597106351284649
8381
2.9250000000000007
8382
3.468733619538242
8383
3.7942845369464115
8384
3.497637690720238
8385
2.7742818889657133
8386
3.623528836754643
8387
3.6754338091501886
8388
3.67846498432358
8389
4.541190832846142
8390
3.9743135041000963
8391
3.4175344908805263
8392
3.3250000000000015
8393
3.478538931377669
8394
4.099257125551929
8395
3.47377641190441
8396
4.4055913424837385
8397
2.993415840890

3.142306798294501
8719
4.559686629870253
8720
3.5631817790179237
8721
2.5247725781776404
8722
2.951259918347718
8723
3.0500000000000003
8724
4.093346890384912
8725
3.900630456006593
8726
2.885199124371554
8727
3.62361268020893
8728
3.935907479661359
8729
4.357326052591139
8730
2.8739439346557747
8731
3.893584657710509
8732
4.232668462795356
8733
3.5252445789617943
8734
3.3447895054740604
8735
3.9432190795151314
8736
3.58480009824033
8737
3.4260636113515295
8738
4.116070569211184
8739
2.9720802327276035
8740
3.6269675189656265
8741
3.986029937154526
8742
3.8994599908024177
8743
3.7958788323154753
8744
2.3778707866395603
8745
3.7644462830644847
8746
3.6489504852010284
8747
3.5851658759526934
8748
3.6361210338838226
8749
2.5206120904398506
8750
4.5702458135728845
8751
2.899487090218165
8752
2.8922023209698664
8753
4.80969322937659
8754
3.969100297415927
8755
3.115657784886508
8756
3.7272727272727275
8757
3.825
8758
2.9879814802319933
8759
3.7750000000000004
8760
3.7157349567228812
8761
3.

3.884034602474431
9082
3.9594656878092422
9083
3.0000000000000018
9084
3.3000000000000003
9085
3.290728556616456
9086
3.5057771986271824
9087
3.7740963141958646
9088
3.6099888213578692
9089
3.6344650881731613
9090
3.5265942270054893
9091
2.423240446286354
9092
4.331385151774785
9093
3.206563788642529
9094
3.7524546071481883
9095
2.6847854299122154
9096
3.3440508765901296
9097
3.864767591912912
9098
3.95059643824301
9099
2.957219067446149
9100
4.354423313661282
9101
3.8191171564504103
9102
3.3793784139234533
9103
3.6958332654169146
9104
3.3660254037844393
9105
3.302117470887808
9106
3.40733894918049
9107
4.089264135344625
9108
3.352094613967734
9109
3.4532529182642633
9110
3.951706113292529
9111
3.4464096015517742
9112
3.3776700509851385
9113
3.849782893222309
9114
2.973049832112332
9115
2.444444444444444
9116
4.202155157366158
9117
3.8769843956626273
9118
3.625000000000001
9119
3.791629134738447
9120
3.7416264178005636
9121
3.6877207004478665
9122
3.7565263992861877
9123
3.704627304227

3.5034615384615386
9444
4.000000000000002
9445
3.025
9446
2.8828217227273822
9447
3.3363419938199512
9448
3.475000000000001
9449
3.4270684098145003
9450
3.2236376921528054
9451
3.0750000000000006
9452
3.1826793932852713
9453
2.7039564538349663
9454
5.0
9455
4.683012701892219
9456
3.5000000000000013
9457
3.578906054719685
9458
4.044518572532058
9459
3.4749999999999996
9460
3.5920501224347845
9461
3.828050635760924
9462
3.6
9463
4.269877280361693
9464
3.529082330697885
9465
3.324967344723245
9466
4.284175552940927
9467
3.400000000000001
9468
3.6221011627261626
9469
3.6221011627261626
9470
3.047665907859118
9471
1.8281940814013922
9472
3.4731397519822504
9473
3.4263697613379653
9474
3.3750000000000018
9475
2.025000000000001
9476
4.249510183084429
9477
2.4722464584806945
9478
4.252268242767146
9479
3.8545443984274623
9480
3.63767107092785
9481
3.6551465541224597
9482
3.471576053651616
9483
2.951170693159976
9484
3.450837415333123
9485
3.0861032234765604
9486
2.8000000000000007
9487
3.52500

3.925000000000001
9804
3.1828911435535567
9805
3.377419142437662
9806
3.15
9807
3.9054166666666665
9808
3.5
9809
4.125000000000001
9810
2.7482338692369845
9811
2.6476701878490605
9812
3.4235885792485945
9813
4.159699176759125
9814
3.8288432272827495
9815
0
9816
3.9783217208648707
9817
1.0
9818
2.8750000000000013
9819
2.6750000000000003
9820
3.833333333333333
9821
3.7446926200008837
9822
2.515395816242822
9823
2.3999999999999995
9824
3.273973657921026
9825
3.273973657921026
9826
2.4360331816236824
9827
3.326019292816036
9828
3.349999999999999
9829
4.106487833915352
9830
3.050337212607387
9831
3.2582664682372693
9832
3.7098138922847377
9833
2.7116902410034647
9834
3.110915200391007
9835
3.9250000000000007
9836
3.428997022573051
9837
2.4564759912167387
9838
3.428997022573051
9839
3.493371943371943
9840
3.0955013733390393
9841
3.1490963203463207
9842
2.9472252673133474
9843
3.3341061935140885
9844
3.4856723487788854
9845
2.7394541073322225
9846
3.189813504645676
9847
3.6540431488801053
984

#### RMSE measure (for k = 20, choose top 20 similar movies)

In [31]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# add all actual ratings to array that can measure the RMSE with rediction ratings
actuals = []
for i in range(0, len(test_array)):
    actuals.append(test_array[i][1])
#print(actuals)
rmse = sqrt(mean_squared_error(actuals, pred)) #sqrt(MSE)
print("RMSE: %f" % rmse)

#compute global average
total = 0
for i in range(0, len(test_array)):
    
    total += test_array[i][1]

# Array where all estimates are the global average global
glbl = []
glbl_avg = total / len(test_array)

for i in range(0, len(test_array)):
    glbl.append(glbl_avg)

#compute RMSE    
rmse1 = sqrt(mean_squared_error(actuals, glbl))
print("Global:", rmse1)

#compute user averages
users = []
for i in range(0, len(test_array)):
    users.append([0,0])

#index [x] is whichever user id we are calculating, sum up all averages and count their reviews
for i in range(0, len(test_array)):
    users[test_array[i][0]][0] += test_array[i][1]
    users[test_array[i][0]][1] += 1

#new array, each index corresponds to a user id, and has their average rating
users_avg = []
for i in range(0, len(test_array)):
    if (users[i][1]) > 0:
        users_avg.append(users[i][0] / users[i][1])
    else:
        users_avg.append(0)

        #compute RMSE
se = 0
for i in range(0, len(test_array)):
    se += (test_array[i][1] - users_avg[test_array[i][0]])**2

mse = se / len(test_array)

rmse2 = sqrt(mse)

print("User Average:", rmse2)

#compute item averages
items = []
for i in range(1, 193610):
    items.append([0,0])

#each index is a movie id, storing total rating and number of reviews
for i in range(0, len(test_array)):
    items[test_array[i][0]][0] += test_array[i][1]
    items[test_array[i][0]][1] += 1

items_avg = []

#Calculate average for each movie
for i in range(0, len(items)):
    if (items[i][1]) > 0:
        items_avg.append(items[i][0] / items[i][1])
    else:
        items_avg.append(0)

#Compute RMSE        
for i in range(0, len(test_array)):
    se += (test_array[i][1] - items_avg[test_array[i][0]])**2

mse = se / len(test_array)

rmse3 = sqrt(mse)
print("Item:", rmse3)

RMSE: 0.971519
Global: 1.053795858633015
User Average: 0.9152171669149258
Item: 1.2943125299677793
